In [71]:
pip install pyvi

In [72]:
import os
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_files
from pyvi import ViTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

%matplotlib inline

In [73]:
!unzip news.zip

Archive:  news.zip
replace news_1135/Giải trí/00a00d8f63505813279de920ee6b0e364fdbf062.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [74]:
INPUT = '/content/news_1135'
os.makedirs("images",exist_ok=True)  # thư mục lưu các các hình ảnh trong quá trình huấn luyện và đánh gía

In [75]:
# statistics
print('Các nhãn và số văn bản tương ứng trong dữ liệu')
print('----------------------------------------------')
n = 0
for label in os.listdir(INPUT):
    print(f'{label}: {len(os.listdir(os.path.join(INPUT, label)))}')
    n += len(os.listdir(os.path.join(INPUT, label)))

print('-------------------------')
print(f"Tổng số văn bản: {n}")

Các nhãn và số văn bản tương ứng trong dữ liệu
----------------------------------------------
Thời sự: 138
Kinh tế: 186
Thể thao: 140
Giải trí: 107
Sức khỏe: 75
Khoa học - Công nghệ: 196
Độc giả: 52
Pháp luật: 50
Đời sống - Xã hội: 91
Tin khác: 100
-------------------------
Tổng số văn bản: 1135


In [76]:
# load data
data_train = load_files(container_path=INPUT, encoding="utf-8")
print('mapping:')
for i in range(len(data_train.target_names)):
    print(f'{data_train.target_names[i]} - {i}')

print('--------------------------')
print(data_train.filenames[0:1])
# print(data_train.data[0:1])
print(data_train.target[0:1])
print(data_train.data[0:1])

print("\nTổng số  văn bản: {}" .format( len(data_train.filenames)))

mapping:
Giải trí - 0
Khoa học - Công nghệ - 1
Kinh tế - 2
Pháp luật - 3
Sức khỏe - 4
Thể thao - 5
Thời sự - 6
Tin khác - 7
Đời sống - Xã hội - 8
Độc giả - 9
--------------------------
['/content/news_1135/Tin khác/0218e1df21ce358b9c6485176a48f1fcaeedef67.txt']
[7]
['Dân_trí Sở GD & ĐT tỉnh Gia_Lai vừa ra văn_bản số 2258 / SGDĐT - VP , về việc chấn_chỉnh việc tiếp_thị sách và các vật_dụng khác trong các cơ_sở giáo_dục . Văn_bản chỉ_đạo , tuyệt_đối không cho phép các cá_nhân , tập_thể đến trường tiếp_thị , quảng_cáo mua_bán sách , dụng_cụ học_tập … cho giáo_viên và học_sinh trong nhà_trường . Các tổ_chức , cá_nhân trong ngành giáo_dục tuyệt_đối không được thực_hiện hoặc tham_gia giới_thiệu , quảng_bá , vận_động mua , phát_hành sách tham_khảo tới học_sinh hoặc phụ_huynh dưới hình_thức nào . Nhà_trường tuyệt_đối không được lưu_hành , sử_dụng sách có nội_dung không lành_mạnh , không phù_hợp với nội_dung chương_trình phổ_thông . Trường_hợp phát_hiện sách có s

In [77]:
# load dữ liệu các stopwords
vn_stopwords = ['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh', 'anh_ấy']

# Chuyển hoá dữ liệu text về dạng vector TF
#     - loại bỏ từ dừng
#     - sinh từ điển

#remove stopwords:
data_train_no_sw = [] #store data cleaned
count = 0
for doc in data_train.data:
  words = doc.split()
  filtered_words = []
  for word in words:
    if word in vn_stopwords:
      count+=1
    else:
      filtered_words.append(word)
  filtered_doc = ' '.join(filtered_words)
  data_train_no_sw.append(filtered_doc)
data_train.data = data_train_no_sw
print(f"so luong stopwords: {count}")

#generate dictionary
vectorizer = CountVectorizer()

# Hàm thực hiện chuyển đổi dữ liệu text thành dữ liệu số dạng ma trận
# Input: Dữ liệu 2 chiều dạng numpy.array, mảng nhãn id dạng numpy.array

data_preprocessed = vectorizer.fit_transform(data_train.data)

X = data_preprocessed # thuoc tinh
Y = data_train.target #nhan

print(f"\nSố lượng từ trong từ điển: {len(vectorizer.vocabulary_)}")
print(f"Kích thước dữ liệu sau khi xử lý: {X.shape}")
print(f"Kích thước nhãn tương ứng: {Y.shape}")

so luong stopwords: 1134

Số lượng từ trong từ điển: 25198
Kích thước dữ liệu sau khi xử lý: (1135, 25198)
Kích thước nhãn tương ứng: (1135,)
